<a href="https://colab.research.google.com/github/aleks-haksly/ClickHouse/blob/main/experiments_preparation/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Импортируем используемые в работе модули
import pandas as pd
!pip install clickhouse-driver --q
from clickhouse_driver import Client
from google.colab import userdata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 935.7/935.7 kB 4.9 MB/s eta 0:00:00


In [2]:
# Реквизиты для подключения к БД
user="hardda_student_alekse-fomin"
password=userdata.get('HA[hardda_student_alekse-fomin]')

host='clickhouse.lab.karpov.courses'
port=9000
database_r='hardda'

In [3]:
# Создаем соединение с БД ClickHouse
client_r = Client(host=host, port=port, user=user,
                password=password, database=database_r)

### Задание 1 ###
Подготовьте SQL запрос для клиентов, совершивших оплату в Москве в период с 11 по 16 октября 2022 г. за услугу "premium".

In [4]:
sql = """WITH transactions as (
  SELECT
    payment_date,
    advert_id,
    amount,
    passport_id
  FROM
    user_transactions t
  WHERE
    t.payment_date BETWEEN '2022-10-11'
    and '2022-10-16'
    AND t.sign = -1
    and t.type = 'premium'
    and t.amount != 0
)

SELECT
  t.passport_id,
  abs(sum(t.amount)) AS revenue
FROM
  transactions AS t
LEFT JOIN
  dp_live_adverts AS a ON CAST(a.advert_id, 'UInt32') = t.advert_id AND a.execution_date = t.payment_date
WHERE
  region = 'Москва'
GROUP BY
  t.passport_id
"""
result, columns = client_r.execute(sql, with_column_types=True)
df = pd.DataFrame(result, columns=[tuple[0] for tuple in columns]) #.set_index('payment_date')
#df['payment_date'] = pd.to_datetime(df['payment_date'], format="yyyy-mm-dd")
#df['tstamp'] = pd.to_datetime(df['tstamp'], format="yyyy-mm-dd hh:mm:ss")

In [5]:
df

,passport_id,revenue
0,140977147,985
1,134882393,1105
2,139901702,1100
3,123673877,1100
4,142445756,1100
...,...,...
1048,124592490,1105
1049,135976735,1100
1050,123466729,1100
1051,142277608,1105


### Задание 2 ###
В продукте планируется внедрение нового функционала с целью улучшить привлекательность услуги "premium". Менеджеры предполагают, что это позволит увеличить среднюю выручку на 8% (поле amount).

Вам необходимо определить минимально необходимый размер выборки, чтобы статистически значимо задетектить эффект при увеличении выручки на 8% с учетом мощности теста в 80% и установленного уровня значимости alpha = 0.05.

Давайте решать эту задачу по шагам. В первую очередь, нам необходимо определить среднюю выручку и стандартное отклонение. И найти ожидаемую менеджером среднюю выручку (ARPPU) с учетом ее увеличения на 8%.

In [6]:
ARPPU = round(df.revenue.mean() * 1.08, 2)
print("Ожидаемая менеджером средняя выручка (ARPPU) с учетом ее увеличения на 8%% составляет %s " % ARPPU)

Ожидаемая менеджером средняя выручка (ARPPU) с учетом ее увеличения на 8% составляет 1754.88 


### Задание 3 ###
Найдите значение для стандартизированного эффекта (индекс Коэна).

In [9]:
cohens_d = (ARPPU - df.revenue.mean()) / df.revenue.std()
print("Значение для стандартизированного эффекта (индекс Коэна) составляет %s " % round(cohens_d, 2))

Значение для стандартизированного эффекта (индекс Коэна) составляет 0.08 


### Задание 4 ###
Используя рассчитанные значения, найдите минимально необходимый размер выборки, чтобы статистически значимо задетектить эффект.
Рассчитываем минимальный размер выборки, необходимый, чтобы стат.значимо задетектить разницу с вероятностью совершить ошибку II рода ниже 20% при уровне значимости альфа = 0.05:

In [10]:
from statsmodels.stats.power import tt_ind_solve_power

In [16]:
sample_size = round(tt_ind_solve_power(effect_size=cohens_d,
                                       nobs1=None, # Его мы и ищем
                                       alpha=0.05,
                                       power=0.8,
                                       ratio=1.0,
                                       alternative='two-sided'))
print('Размер требуемой выборки для каждой группы не менее: %s' % sample_size)

Размер требуемой выборки для каждой группы не менее: 2369


2368.676655159133